In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))


In [2]:
def parse_edge(s):
  user, follower = s.split("\t")
  return (int(user), int(follower))

def step(item):
  prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
  return (next_v, prev_d + 1)

def complete(item):
  v, old_d, new_d = item[0], item[1][0], item[1][1]
  return (v, old_d if old_d is not None else new_d)

def step_count(item):
    prev_v, ((prev_d, visited), next_v) = item
    return (next_v, (prev_d+1, visited+(next_v,)))
    

In [3]:
n = 4  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

In [32]:
x = 12
d = 0
distances = sc.parallelize([(x, d)]).partitionBy(n)
distances_route = sc.parallelize([(x, (d,(12,)))]).partitionBy(n)

In [33]:
while True:
    candidates_stop = distances_route.join(forward_edges, n).map(step_count)
    new_distances_path= distances_route.fullOuterJoin(candidates_stop,n).map(complete, True).persist()
    count = new_distances_path.filter(lambda i: i[1][0] == d+1).count()
    if count > 0:
        d += 1
        distances_route = new_distances_path
        print("d = ", d, "count = ", count)
    else:
        break


('d = ', 1, 'count = ', 4)
('d = ', 2, 'count = ', 1)
('d = ', 3, 'count = ', 7)
('d = ', 4, 'count = ', 2)
('d = ', 5, 'count = ', 3)
('d = ', 6, 'count = ', 3)
('d = ', 7, 'count = ', 4)
('d = ', 8, 'count = ', 2)


In [34]:
result =distances_route.filter(lambda x: x[0]==34).collect()

In [35]:
result[0][1][1]

(12, 422, 53, 52, 107, 20, 23, 274, 34)

In [38]:
print ",".join(str(x) for x in result[0][1][1])

'12,422,53,52,107,20,23,274,34'